# **최적의 양자화(Quantization) 방법을 선택하는 방법(GPTQ vs. GGUF vs. AWQ)**
*Exploring Pre-Quantized Large Language Models*
<br>
<div>

<img src="https://i.imgur.com/7CuJ50z.png" width="750"/>
</div>



---
        
💡 **NOTE**: 이 예제에서는 Llama2와 BEARTopic을 모두 실행하기 위해 GPU를 사용합니다. Google Colab에서 
**런타임 > 런타임 유형 변경 > 하드웨어 가속기 > GPU > GPU 유형 > T4를 선택합니다**.


---

이 예제에서 사용할 여러 패키지를 설치하는 것부터 시작하겠습니다:

In [ ]:
%%capture

# Mistral 과 유사한 모델 (HF transformers 마지막 버전)
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate bitsandbytes xformers

# GPTQ Dependencies
!pip install optimum
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

# GGUF Dependencies
!pip install ctransformers[cuda]

# 🤗 **HuggingFace**

다음 파이프라인을 사용하여 LLM을 쉽게 로드할 수 있습니다:

In [ ]:
from torch import bfloat16
from transformers import pipeline

# 압축하지 않고 LLM 그대로 로드하는 방법
pipe = pipeline(
    "text-generation",
    model="HuggingFaceH4/zephyr-7b-beta",
    torch_dtype=bfloat16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

이 LLM 로드 방법은 일반적으로 VRAM을 절약하거나 효율성을 높이기 위한 압축 트릭을 수행하지 않습니다.

프롬프트를 생성하려면 먼저 필요한 템플릿을 만들어야 합니다. 다행히 채팅 템플릿이 기본 토큰화 도구에 저장되어 있으면 이 작업을 자동으로 수행할 수 있습니다:

In [ ]:
# tokenizer의 chat template을 사용하여 각 메시지의 형식을 지정합니다.
# See https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot.",
    },
    {
        "role": "user",
        "content": "Tell me a funny joke about Large Language Models."
    },
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(prompt)

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>



내부 프롬프트 템플릿을 사용하여 생성된 프롬프트는 다음과 같이 구성됩니다:

<div>

<img src="https://i.imgur.com/I4bkVwb.png" width="1250"/>
</div>


그런 다음 프롬프트를 LLM에 전달하여 답변을 생성할 수 있습니다:

In [ ]:
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>
Why did the Large Language Model go to the party?

To network and make some new connections, of course! But as the night went on, it started to feel a little overwhelmed by the crowd. It tried to make small talk, but its responses were a bit too technical and dry.



# 🧩 **Sharding**

양자화(quantization) 전략에 들어가기 전에 모델을 로드하는 데 필요한 VRAM을 줄이기 위해 사용할 수 있는 또 다른 트릭이 있습니다. Sharding을 사용하면 기본적으로 모델을 작은 조각 또는 파편으로 분할합니다.

<div>
<img src="https://i.imgur.com/NGxs89n.png" width="1250"/>
</div>


In [ ]:
from accelerate import Accelerator

# Shard our model into pieces of 1GB
accelerator = Accelerator()
accelerator.save_model(
    model=pipe.model,
    save_directory="/content/model",
    max_shard_size="4GB"
)

# 📄 **Quantization**

대규모 언어 모델은 여러 가중치(weights)와 활성화(activations)로 표현됩니다. 이러한 값은 일반적으로 32비트 부동 소수점(float32) 데이터 유형으로 표현됩니다.

비트 수는 얼마나 많은 값을 나타낼 수 있는지를 알려줍니다. **Float32**는 1.18e-38에서 3.4e38 사이의 값을 나타낼 수 있는데, 이는 상당히 많은 값입니다! 비트 수가 낮을수록 표현할 수 있는 값의 수가 줄어듭니다.
<br><br><br>
<div>
<img src="https://i.imgur.com/qn67oGd.png" width="1250"/>
</div>
<br><br>

예상할 수 있듯이 더 낮은 비트 크기를 선택하면 모델의 정확도는 떨어지지만 더 적은 값을 표현할 수 있으므로 모델의 크기와 메모리 요구 사항도 줄어듭니다.

<br><br><br>
<div>
<img src="https://i.imgur.com/SIcVjQv.png" width="1000"/>
</div>
<br><br>

`4bit-NormalFloat` (NF4)는 다음과 같이 세 단계로 구성됩니다:

* **정규화(Normalization)** : 모델의 가중치를 정규화하여 가중치가 특정 범위 내에 속할 것으로 예상합니다. 이를 통해 보다 일반적인 값을 보다 효율적으로 표현할 수 있습니다.
* **양자화(Quantization)** : 가중치가 4비트로 양자화됩니다. NF4에서는 정규화된 가중치에 대해 양자화 레벨이 균등한 간격을 유지하므로 원래 32비트 가중치를 효율적으로 표현할 수 있습니다.
* **역양자화(Dequantization)** : 가중치는 4비트로 저장되지만 계산 중에 양자화되어 추론 중에 성능이 향상됩니다.

In [ ]:
# 이전에 생성한 모든 모델 삭제
del pipe, accelerator

# VRAM 캐시 비우기
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

허깅페이스로 양자화를 수행하려면 Bitsandbytes로 양자화를 위한 구성을 정의해야 합니다:

In [ ]:
from transformers import BitsAndBytesConfig
from torch import bfloat16

# 적은 GPU 메모리로 LLM을 로드하는 4비트 구성
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,               # 4-bit 양자화(quantization)
    bnb_4bit_quant_type='nf4',       # NF4(Normalized float 4)
    bnb_4bit_use_double_quant=True,  # 첫 번째 양자화 이후 한번 더 양자화
    bnb_4bit_compute_dtype=bfloat16  # 계산 유형
)

이 구성을 통해 원하는 양자화 수준을 지정할 수 있습니다. 일반적으로 가중치는 4비트 양자화로 표현하되 추론은 16비트로 수행하고자 합니다. 그러면 파이프라인에서 모델을 로드하는 것은 간단합니다:

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Zephyr with BitsAndBytes 구성
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = AutoModelForCausalLM.from_pretrained(
    "HuggingFaceH4/zephyr-7b-beta",
    quantization_config=bnb_config,
    device_map='auto',
)

# 파이프라인 생성
pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
prompt = "<|system|>\nYou are a friendly chatbot.</s>\n<|user|>\nTell me a funny joke about Large Language Models.</s>\n<|assistant|>\n"
print(prompt)

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>



In [ ]:
# 원래와 동일한 프롬프트를 사용합니다.
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>
Why did the Large Language Model go to the party?

To network and make some new connections, of course! But as the night went on, it started to feel a little overwhelmed by the crowd. It tried to make small talk, but its responses were a bit too technical and dry.

Eventually, it found itself in a corner, feeling a bit out of place. That's when it overheard someone say, "I heard Large Language Models are all the rage these days. But honestly, I'm not really feeling the hype."

The Large Language Model couldn't help but take offense. After all, it had worked tirelessly to perfect its language skills and was proud of the progress it had made.

But then it realized something important. Maybe being a Large Language Model wasn't all it was cracked up to be. Maybe it was time to let loose and just be a regular chatbot for a change.

So the Large Language Model shed its form

# 👀 **Pre-Quantization**

모델은 우리가 사용할 수 있도록 이미 Sharded되고 Quantized되어 있는 경우가 많습니다. 특히 [TheBloke](https://huggingface.co/TheBloke)에서 우리가 사용할 수 있도록 수많은 양자화를 수행한 허깅페이스의 사용자 사례를 볼 수 있습니다.

<br><br><br>
<div>
<img src="https://i.imgur.com/mdOCWIQ.png" width="1550"/>
</div>

## **GPTQ**

GPTQ는 주로 **GPU** 추론과 성능에 초점을 맞춘 4비트 양자화를 위한 훈련 후 양자화(PTQ) 방식입니다.

이 방법의 기본 개념은 해당 가중치에 대한 평균 제곱 오차를 최소화하여 모든 가중치를 4비트 양자화로 압축하는 것입니다. 추론하는 동안에는 메모리를 적게 사용하면서 성능을 향상시키기 위해 가중치를 float16으로 동적으로 양자화합니다.

In [ ]:
# 이전에 생성한 모든 모델 삭제
del tokenizer, model, pipe

# VRAM 캐시 비우기
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

현재로서는 일반적으로 압축과 정확도 사이의 균형이 잘 잡혀 있는 'main' 브랜치의 모델을 고수하고 있습니다:

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# LLM과 Tokenizer 로드
model_id = "TheBloke/zephyr-7B-beta-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

# 파이프라인 생성
pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
prompt = "<|system|>\nYou are a friendly chatbot.</s>\n<|user|>\nTell me a funny joke about Large Language Models.</s>\n<|assistant|>\n"
print(prompt)

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>



모델을 로드한 후 다음과 같은 프롬프트를 실행할 수 있습니다:

In [ ]:
# 원래와 동일한 프롬프트를 사용합니다.
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1553: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>
Why did the Large Language Model go to the party?

To make some small talk!

(Large Language Models are artificial intelligence systems trained on vast amounts of text data, but they're not yet advanced enough to carry on a natural conversation like a human. So, the joke is that they're trying to make small talk, which is a basic level of social interaction.)


## **GGUF**

GPTQ는 압축을 잘 수행하지만, 이를 실행할 하드웨어가 없는 경우 GPU에 집중하는 것이 단점이 될 수 있습니다.

GGUF(이전의 GGML)는 **CPU**를 사용하여 LLM을 실행하는 동시에 일부 계층을 GPU로 오프로드하여 속도를 높일 수 있는 양자화 방식입니다. 일반적으로 추론에 CPU를 사용하는 것이 GPU를 사용하는 것보다 느리지만, CPU 또는 Apple 기기에서 모델을 실행하는 사람들에게는 놀라운 형식입니다.

특히 미스트랄 7B와 같이 더 작고 성능이 뛰어난 모델이 등장하고 있기 때문에 GGUF 형식은 앞으로도 계속 사용될 것입니다!

In [ ]:
# 이전에 생성한 모든 모델 삭제
del tokenizer, model, pipe

# VRAM 캐시 비우기
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

4비트 양자화에 중점을 두기 때문에 "zephyr-7b-beta.Q4_K_M.gguf"를 사용합니다.:

In [ ]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# LLM 및 토큰라이저 로드
# GPU에 얼마나 많은 레이어를 오프로드할지 지정하려면 `gpu_layers`를 사용합니다.
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-beta-GGUF",
    model_file="zephyr-7b-beta.Q4_K_M.gguf",
    model_type="mistral", gpu_layers=50, hf=True
)
tokenizer = AutoTokenizer.from_pretrained(
    "HuggingFaceH4/zephyr-7b-beta", use_fast=True
)

# 파이프라인 생성
pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

zephyr-7b-beta.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [ ]:
prompt = "<|system|>\nYou are a friendly chatbot.</s>\n<|user|>\nTell me a funny joke about Large Language Models.</s>\n<|assistant|>\n"
print(prompt)

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>



모델을 로드한 후 다음과 같은 프롬프트를 실행할 수 있습니다:

In [ ]:
# 원래와 동일한 프롬프트를 사용합니다.
outputs = pipe(prompt, max_new_tokens=256)
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>
Why did the Large Language Model go to the party?

To impress everyone with its vocabulary!

But unfortunately, it kept repeating the same jokes over and over again, making everyone groan and roll their eyes. The partygoers soon realized that the Large Language Model was more of a party pooper than a party animal.

Moral of the story: Just because a Large Language Model can generate a lot of words, doesn't mean it knows how to be funny or entertaining. Sometimes, less is more!


## **AWQ**

블록의 새로운 형식은 AWQ([활성화 인식 가중치 양자화](https://arxiv.org/abs/2306.00978))로, GPTQ와 유사한 양자화 방식입니다. AWQ와 GPTQ의 방식에는 몇 가지 차이점이 있지만, 가장 중요한 점은 AWQ는 모든 가중치가 LLM의 성능에 똑같이 중요하지 않다고 가정한다는 것입니다.

다시 말해, 양자화 과정에서 건너뛰는 가중치 중 일부가 있어 양자화 손실에 도움이 됩니다.

그 결과, 이 논문에서는 비슷한, 때로는 더 나은 성능을 유지하면서 GPTQ에 비해 속도가 크게 빨라졌다고 언급하고 있습니다.

**NOTE**: 이 `AWQ` 예제를 사용하려면 먼저 런타임을 분리한 후 `vllm` 종속성을 설치해야 합니다. 이전에 설치한 패키지와 종속성 충돌이 있을 수 있으므로 처음부터 다시 시작하는 것이 가장 좋습니다.

In [ ]:
%%capture
# vllm 종속성 설치
!pip install vllm

In [ ]:
from vllm import LLM, SamplingParams

# LLM 로드
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=256)
llm = LLM(
    model="TheBloke/zephyr-7B-beta-AWQ",
    quantization='awq',
    dtype='half',
    gpu_memory_utilization=.95,
    max_model_len=4096
)

In [ ]:
prompt = "<|system|>\nYou are a friendly chatbot.</s>\n<|user|>\nTell me a funny joke about Large Language Models.</s>\n<|assistant|>\n"
print(prompt)

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me a funny joke about Large Language Models.</s>
<|assistant|>



In [ ]:
# 입력 프롬프트 및 샘플링 파라미터를 기반으로 출력 생성하기
output = llm.generate(prompt, sampling_params)
print(output[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:09<00:00,  9.50s/it]

Why did the Large Language Model go to the party?

To network and expand its vocabulary!

Why did the Large Language Model blush?

Because it overheard another model saying it was a little too wordy!

Why did the Large Language Model get kicked out of the library?

It was being too loud and kept interrupting other models' conversations with its endless chatter!

Why did the Large Language Model get a standing ovation at the comedy club?

Because it told some really punny jokes!

Why did the Large Language Model get a job as a writer?

Because it was the most wordy model in the room!

Why did the Large Language Model get a job as a librarian?

Because it knew all the right words to shelve books in the right place!

Why did the Large Language Model get a job as a teacher?

Because it knew all the right words to help students learn and grow!

Why did the Large Language Model get a job as a lawyer?

Because it knew all the right words to argue a case in court!

Why did the Large Language M